In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score

## Running KNN only on the first dimension

In [31]:
datasets = [
"CWRU_12k_FE_multivar"
]

datasets_path = "../datasets"

for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy", mmap_mode='r')
    Dataset = Dataset
    print(Dataset.shape)
    
    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy", mmap_mode='r')
    Labels = Labels.squeeze()

(5400, 1000, 3)


In [14]:
Dataset1 = Dataset[:,:,0] #only the first dimension

In [16]:
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Dataset1, Labels, test_size=0.3, random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3780, 1000), (1620, 1000), (3780,), (1620,))

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((2646, 1000), (1134, 1000), (2646,), (1134,))

In [18]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
classifier = KNeighborsTimeSeriesClassifier(distance="euclidean", n_jobs=10)
classifier.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier(distance='euclidean', n_jobs=10)

In [22]:
print("Validation Accuracy:{:.2f}".format(classifier.score(X_val, y_val)))

Validation Accuracy:0.33


In [24]:
print("Test Accuracy:{:.2f}".format(classifier.score(X_test, y_test)))

Test Accuracy:0.33


## Running KNN over different dimensions and ensemble the result

In [28]:
from sklearn import model_selection
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from mlxtend.classifier import EnsembleVoteClassifier

def train_and_evaluate_classifiers(X_train, y_train, X_val, y_val, X_test, y_test, num_classifiers):
    classifiers = []
    labels = []
    train_data = []
    val_data = []

    # Create individual classifiers and add them to the ensemble
    for i in range(num_classifiers):
        print(i)
        clf = KNeighborsTimeSeriesClassifier(distance="euclidean", n_jobs=10)
        classifiers.append(clf)
        labels.append(f'Classifier {i+1}')
        train_data.append((X_train[:,:,i], y_train))
        val_data.append((X_val[:,:,i],y_val))


    # Train and evaluate classifiers
    for clf, label, data , valdata in zip(classifiers, labels, train_data, val_data):
        X_train_part, y_train_part = data
        X_val_part , y_val_part = valdata
        clf.fit(X_train_part, y_train_part)
        print("Validation Accuracy: %0.2f [%s]" % (clf.score(X_val_part, y_val_part), label))
        
    # Add ensemble classifier
    eclf = EnsembleVoteClassifier(clfs=classifiers, voting='soft', weights=[1] * len(classifiers)) # Modified this line

    # Fit the ensemble classifier with training data
    eclf.fit(X_train, y_train)

    # Calculate and print the test accuracy
    print("Test Accuracy: %0.2f" % eclf.score(X_test, y_test))

In [25]:
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Dataset, Labels, test_size=0.3, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

print('Train/Valid/Test sizes:', y_train.shape[0], y_val.shape[0], y_test.shape[0])
X_train.shape, X_val.shape, y_train.shape, y_val.shape

Train/Valid/Test sizes: 2646 1134 1620


((2646, 1000, 3), (1134, 1000, 3), (2646,), (1134,))

In [29]:
# Example usage
num_classifiers = Dataset.shape[2]
print(num_classifiers)# Set the desired number of classifiers

3


In [30]:
train_and_evaluate_classifiers(X_train, y_train, X_val, y_val, X_test, y_test, num_classifiers)

0
1
2
Validation Accuracy: 0.33 [Classifier 1]
Validation Accuracy: 0.36 [Classifier 2]
Validation Accuracy: 0.74 [Classifier 3]
Test Accuracy: 0.56


In [32]:
(0.33 + 0.36 + 0.74)/3

0.4766666666666666

In [33]:
datasets = [
# "Gas_sensors_home_activity",
"PHM2022_Multivar",

]

datasets_path = "../datasets"

for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    Dataset = Dataset
    print(Dataset.shape)
    
    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")
    Labels = Labels.squeeze()

(53785, 749, 3)


In [34]:
# Example usage
num_classifiers = Dataset.shape[2]
print(num_classifiers)# Set the desired number of classifiers

3


In [35]:
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Dataset, Labels, test_size=0.3, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

print('Train/Valid/Test sizes:', y_train.shape[0], y_val.shape[0], y_test.shape[0])

Train/Valid/Test sizes: 26354 11295 16136


In [36]:
train_and_evaluate_classifiers(X_train, y_train, X_val, y_val, X_test, y_test, num_classifiers)

0
1
2
Validation Accuracy: 0.94 [Classifier 1]
Validation Accuracy: 0.90 [Classifier 2]
Validation Accuracy: 0.91 [Classifier 3]
Test Accuracy: 0.97


In [ ]:
#pip install mlxtend

In [ ]:
from mlxtend.classifier import EnsembleVoteClassifier


In [ ]:
from sklearn import model_selection
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier


clf1 = KNeighborsTimeSeriesClassifier(distance= "euclidean", n_jobs=10)
clf2 = KNeighborsTimeSeriesClassifier(distance= "euclidean", n_jobs=10)
clf3 = KNeighborsTimeSeriesClassifier(distance= "euclidean", n_jobs=10)
eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], weights=[1, 1, 1])

labels = ['Classifier 1', 'Classifier 2', 'Classifier 3', 'Ensemble']

# Split the training data into three subsets
X_train1, X_train2, X_train3 = X_train[:,:,0], X_train[:,:,1], X_train[:,:,2]

classifiers = [clf1, clf2, clf3, eclf]
train_data = [(X_train1, y_train), (X_train2, y_train), (X_train3, y_train)]

for clf, label, data in zip(classifiers, labels, train_data):
    X_train_part, y_train_part = data
    clf.fit(X_train_part, y_train_part)
    print("Validation Accuracy: %0.2f [%s]" % (clf.score(X_val, y_val), label))
    
eclf.fit(X_train, y_train)
print("Test Accuracy: %0.2f" % eclf.score(X_test, y_test))


In [ ]:
from sklearn import model_selection
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier


clf1 = KNeighborsTimeSeriesClassifier(distance= "euclidean", n_jobs=10)
clf2 = KNeighborsTimeSeriesClassifier(distance= "euclidean", n_jobs=10)
clf3 = KNeighborsTimeSeriesClassifier(distance= "euclidean", n_jobs=10)
eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], voting='soft', weights=[1, 1, 1])

labels = ['Classifier 1', 'Classifier 2', 'Classifier 3', 'Ensemble']

# Split the training data into three subsets
X_train1, X_train2, X_train3 = X_train[:,:,0], X_train[:,:,1], X_train[:,:,2]

classifiers = [clf1, clf2, clf3, eclf]
train_data = [(X_train1, y_train), (X_train2, y_train), (X_train3, y_train)]

for clf, label, data in zip(classifiers, labels, train_data):
    X_train_part, y_train_part = data
    clf.fit(X_train_part, y_train_part)
    print("Validation Accuracy: %0.2f [%s]" % (clf.score(X_val, y_val), label))
    
eclf.fit(X_train, y_train)
print("Test Accuracy: %0.2f" % eclf.score(X_test, y_test))

In [ ]:
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = \
    train_test_split(Dataset, Labels, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=1)

print('Train/Valid/Test sizes:', y_train.shape[0], y_val.shape[0], y_test.shape[0])
print(X_train.shape)
print(Labels.shape)

In [ ]:
from sklearn import model_selection
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from mlxtend.classifier import EnsembleVoteClassifier

def train_and_evaluate_classifiers(X_train, y_train, X_val, y_val, X_test, y_test, num_classifiers):
    classifiers = []
    labels = []
    train_data = []

    # Create individual classifiers and add them to the ensemble
    for i in range(num_classifiers):
        print(i)
        clf = KNeighborsTimeSeriesClassifier(distance="euclidean", n_jobs=10)
        classifiers.append(clf)
        labels.append(f'Classifier {i+1}')
        train_data.append((X_train[:,:,i], y_train))

    # Add ensemble classifier
    eclf = EnsembleVoteClassifier(clfs=classifiers, voting='soft', weights=[1] * num_classifiers)
    classifiers.append(eclf)
    labels.append('Ensemble')
    train_data.append((X_train, y_train))

    # Train and evaluate classifiers
    for clf, label, data in zip(classifiers, labels, train_data):
        X_train_part, y_train_part = data
        clf.fit(X_train_part, y_train_part)
        print("Validation Accuracy: %0.2f [%s]" % (clf.score(X_val, y_val), label))

    # Fit the ensemble classifier with training data
    eclf.fit(X_train, y_train)

    # Calculate and print the test accuracy
    print("Test Accuracy: %0.2f" % eclf.score(X_test, y_test))




In [ ]:
def train_and_evaluate_classifiers(X_train, y_train, X_val, y_val, X_test, y_test, num_classifiers):
    classifiers = []
    labels = []
    train_data = []

    # Create individual classifiers and add them to the ensemble
    for i in range(num_classifiers):
        print(i)
        clf = KNeighborsTimeSeriesClassifier(distance="euclidean", n_jobs=10)
        classifiers.append(clf)
        labels.append(f'Classifier {i+1}')
        train_data.append((X_train[:,:,i], y_train))

    # Add ensemble classifier
    eclf = EnsembleVoteClassifier(clfs=classifiers, voting='soft', weights=[1] * len(classifiers)) # Modified this line
    classifiers.append(eclf)
    labels.append('Ensemble')
    train_data.append((X_train, y_train))

    # Train and evaluate classifiers
    for clf, label, data in zip(classifiers, labels, train_data):
        X_train_part, y_train_part = data
        clf.fit(X_train_part, y_train_part)
        print("Validation Accuracy: %0.2f [%s]" % (clf.score(X_val, y_val), label))

    # Fit the ensemble classifier with training data
    eclf.fit(X_train, y_train)

    # Calculate and print the test accuracy
    print("Test Accuracy: %0.2f" % eclf.score(X_test, y_test))

In [ ]:
# Example usage
num_classifiers = Dataset.shape[2]
print(num_classifiers)# Set the desired number of classifiers

In [ ]:
train_and_evaluate_classifiers(X_train, y_train, X_val, y_val, X_test, y_test, num_classifiers)

In [ ]:
print(f"\n The dataset shape is:{Dataset.shape}")
N=Dataset.shape[0]
print(f"\n The number of data samples (N) is:{N}")

T=Dataset.shape[1]
print(f"\n The number of TS length (T) is:{T}")

M=Dataset.shape[2]
print(f"\n The number of TS dimention (M) is:{M}")

In [ ]:
# split the multivariate time-series into separate univariate time-series, one for each variable
X_univariate = np.split(Dataset, M, axis=2)
len(X_univariate)

In [ ]:
X_univariate[0]

In [ ]:
Dataset[:,:,0].shape

In [ ]:
# list to store predicted class labels for each univariate time-series
y_pred_univariate = []

# perform KNN classification on each univariate time-series independently
for X_var in X_univariate:
    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_var.reshape(N, T), Labels.squeeze(), test_size=0.2, shuffle=True)
    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    # KNN classification
    from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
    classifier = KNeighborsTimeSeriesClassifier(distance= "euclidean", n_jobs=10)
    classifier.fit(X_train, y_train)
    
        
    # predict class labels
    y_pred_var = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_var)
    print(f"accuracy for dimension:{accuracy}")
    y_pred_univariate.append(y_pred_var)
    print(len(y_pred_univariate))
    


In [ ]:
np.unique(Labels)

In [ ]:
y_pred_univariate

In [ ]:
y_pred_univariate[2][0]

In [ ]:
# ensemble the results by taking the majority vote for each time step
#y_pred_ensemble = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=y_pred_univariate)
y_pred_ensemble = np.apply_along_axis(lambda x: np.argmax(np.bincount(np.round(x).astype(int))), axis=0, arr=y_pred_univariate)


In [ ]:
y_pred_ensemble

In [ ]:
# compute the accuracy of the final classification
accuracy = accuracy_score(y_test, y_pred_ensemble)

f1 = f1_score(y_test, y_pred_ensemble, average='weighted')
print(f1)

print("Accuracy of KNN classification on multivariate time-series:", accuracy)


In [38]:
import time
time.localtime(1683369995.3284798)

time.struct_time(tm_year=2023, tm_mon=5, tm_mday=6, tm_hour=6, tm_min=46, tm_sec=35, tm_wday=5, tm_yday=126, tm_isdst=1)

In [39]:
13400.2710/3600

3.7222975000000003